In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join network example 5 - similar to example_forkJoin_1 but closed
val model = Network("model")

// Create network
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 5, delay, 0)

In [ ]:
// Service configurations
delay.setService(jobclass1, Exp(1.0))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)
model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))

// Add MVA with specific options for fork-join
solvers.add(MVA(model, "fork_join", "ht", "method", "amva"))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This basic closed fork-join model demonstrates:

1. **Symmetric Fork-Join Structure**:
   - Jobs fork into two identical parallel branches
   - Queue1 and Queue2 have identical service rates (1.0)
   - Both branches use PS (Processor Sharing) scheduling
   - Synchronization at the Join node

2. **Closed System Properties**:
   - 5 jobs circulate in the system
   - Jobs cycle: Delay → Fork → {Queue1, Queue2} → Join → Delay
   - Fixed population maintains system load

3. **Service Configuration**:
   - All stations have exponential service with rate 1.0
   - Symmetric design ensures balanced load distribution
   - PS scheduling provides fair sharing within each queue

4. **Fork-Join Behavior**:
   - Fork node splits jobs into parallel tasks
   - Each job creates two parallel tasks (one per branch)
   - Join node waits for both tasks to complete
   - Synchronization creates dependencies between branches

5. **Solver Methods**:
   - **JMT**: Discrete-event simulation
   - **MVA**: Analytical solution with fork-join extensions
     - Uses "ht" (Heidelberger-Trivedi) method for fork-join
     - AMVA (Approximate MVA) for computational efficiency

The model represents:
- Parallel processing systems
- Manufacturing with parallel workstations
- Computer systems with parallel execution units
- Service systems with parallel service channels

Key insight: Even with symmetric branches, fork-join synchronization creates complex queueing behavior due to the need to wait for the slower of the two parallel tasks.